# Figure S4: Mean thermocline state and standard deviation observation vs. model

1) Evalutate **first cell** to import necessary libraries und functions  
2) Scroll down to **Plot Figure S4** to load data for Figure 3 and plotting code


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches
import cmocean as cmo
import numpy as np
import cartopy.crs as ccrs
import cartopy
import pandas as pd
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from scipy.interpolate import griddata
from scipy.io import loadmat
import datetime
import string

import sys
sys.path.append("./")  # adds upper level to working directory\n
from utils_iohc_ummenhofer2020 import finished_plot


# where to save plots
plotsave = 'plots/'
datapath = '/vortexfs1/share/clidex/data/'

# Prepare data

In [ ]:
#############################################
# mean and std of thermocline IO
run = 'k003'

# load data
if run == 'k003':
    thermocline = xr.open_dataset('/vortexfs1/share/clidex/data/ORCA/ORCA025.L46.LIM2vp.CFCSF6.JRA.XIOS2-K003.hindcast/'\
                                  'derived_fields/k003_therm20_int1m_180E_180W_50S_45N.nc')
elif run =='k004':
    thermocline = xr.open_dataset('/vortexfs1/share/clidex/data/ORCA/ORCA025.L46.LIM2vp.CFCSF6.JRA.XIOS2-K004.thermhal90/'\
                                  'derived_fields/k004_therm20_int1m_180E_180W_50S_45N.nc')
if run == 'k005':
    thermocline = xr.open_dataset('/vortexfs1/share/clidex/data/ORCA/ORCA025.L46.LIM2vp.CFCSF6.JRA.XIOS2-K005.wind90/'\
                                  'derived_fields/k005_therm20_int1m_180E_180W_50S_45N.nc')
    
#####################################################
# Define baseline for plots
base=['1960','2016']

# mask areas in Maritime Continent where isotherm not always present
dummy = thermocline['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=False).values
mask = np.zeros(thermocline['nav_lon'].shape)
wo = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-22) & (thermocline['nav_lon'].values<150) & 
               (thermocline['nav_lat'].values<25))
wo2 = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-24) & (thermocline['nav_lon'].values>=140) & 
               (thermocline['nav_lon'].values<=155) & (thermocline['nav_lat'].values<25))
mask[wo] = 1000
mask[wo2] = 1000


####################################################
# time-mean map
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline.nav_lon,thermocline.nav_lat,
                   thermocline['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=True).where(mask<1000),vmin=0,vmax=250,
                   cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='depth [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

# plt.savefig(plotsave + run + '_isotherm20_time_mean_'+base[0]+'_'+base[1]+'_masked_MC.png',dpi=300,bbox_inches='tight')


#####################################################
# standard deviation
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline.nav_lon,thermocline.nav_lat,
                   thermocline['therm20'].sel(time_counter=slice(*base)).std('time_counter',skipna=True).where(mask<1000),vmin=0,vmax=45,
                   cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='standard deviation [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

# plt.savefig(plotsave + run + '_isotherm20_std_'+base[0]+'_'+base[1]+'_masked_MC.png',dpi=300,bbox_inches='tight')

## Thermocline detection in Ishii and ISAS 15

In [ ]:
#####################################################
# extract isotherm ishii

# function 
def calc_iso_surface(my_array, my_value, zs, interp_order=6, power_parameter=2):
    """
        - axis for interpolation, e.g. depth has to be first
        - should work for multidimensions as long as depth is first dimension
        - inputs have to be numpy arrays    
        
        ! Still not sure what power parameter does and I think weighting becomes almost
        unnecessary when data is interpolated in vertical first!
    """
    if interp_order < 1: interp_order = 1
    # derive squared difference
    dist = ((my_array - my_value)**2)
    # get index for sorted array in depth axis
    arg = np.argsort(dist,axis=0)
    # sort along depth
    dist.sort(axis=0)
    w_total = 0.
    # create output array
    z = np.zeros(my_array.shape[1:], dtype=float)
    for i in range(int(interp_order)):
        # depth index of value closest to target
        zi = np.take(zs, arg[i,::])
        # associated squared difference value
        valuei = dist[i,::]
        # weighting
        wi = 1/valuei
        np.clip(wi, 0, 1.e6, out=wi) # avoiding overflows
        w_total += wi**power_parameter
        z += zi*wi**power_parameter
    z /= w_total
    return z,dist[0,::]

# load data
ishii = xr.open_dataset(datapath + 'obs/Ishii/ishii_20E_180E_72S_35N.nc').sel(lat=slice(-45,35))


val = 20  # 20degC isotherm
thres=0.2 # cuts off every larger than 20 pm 0.2 degC
arraysize = [1]
for i in range(len(ishii['var80'][0,0,::].shape)):
    arraysize.append(list(ishii['var80'][0,0,::].shape)[i])
iso20 = np.empty((arraysize))
for i in range(ishii['var80'].shape[0]):
    dummy,dist_int = calc_iso_surface(ishii['var80'][i,::].interp(depth=np.arange(0,300,1),method='linear').values,
                                       val,np.arange(0,300,1),interp_order=1)
    dummy[dist_int>thres**2] = np.nan
    iso20 = np.concatenate((iso20,dummy[None,...]),axis=0)
    if i % 50 == 0: print('time step: ' + str(i))

# remove first empty array
iso20 = iso20[1:,::]

# create xarray
thermocline_ishii = xr.DataArray(iso20,coords={"time": ishii.time.values,
                           "lon": (('lon'),ishii.lon),
                           "lat": (('lat'),ishii.lat)},
             dims=["time", "lat","lon"])

# save to netcdf
thermocline_ishii.to_dataset(name='therm20').to_netcdf('/vortexfs1/share/clidex/data/obs/Ishii/ishii_therm20_int1m_20E_180E_45S_35N.nc')

In [ ]:
#############################################
# mean and std of thermocline IO

thermocline_ishii = thermocline_ishii.to_dataset(name='therm20')
# base = ['1960','2012']
base=['2002','2012']


# time-mean map
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline_ishii.lon,thermocline_ishii.lat,
                   thermocline_ishii['therm20'].sel(time=slice(*base)).mean('time',skipna=True),vmin=0,vmax=250,
                   cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='depth [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

# plt.savefig(plotsave + 'ishii_'+base[0]+'_'+base[1]+'_isotherm20_time_mean.png',dpi=300,bbox_inches='tight')


# standard deviation
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline_ishii.lon,thermocline_ishii.lat,
                   thermocline_ishii['therm20'].sel(time=slice(*base)).std('time',skipna=True),vmin=0,vmax=45,
                   cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='standard deviation [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

# plt.savefig(plotsave + 'ishii_'+base[0]+'_'+base[1]+'_isotherm20_std.png',dpi=300,bbox_inches='tight')

### ISAS15

In [ ]:
argo = xr.open_dataset(datapath+'obs/ARGO/ISAS15/isas15_temp_30E_160W_40S_35N.nc')

val = 20
thres=0.2
arraysize = [1]
for i in range(len(argo['TEMP'][0,0,::].shape)):
    arraysize.append(list(argo['TEMP'][0,0,::].shape)[i])
iso20 = np.empty((arraysize))
for i in range(argo['TEMP'].shape[0]):
    dummy,dist_int = calc_iso_surface(argo['TEMP'][i,::].interp(depth=np.arange(0,300,1),method='linear').values,
                                       val,np.arange(0,300,1),interp_order=1)
    dummy[dist_int>thres**2] = np.nan
    iso20 = np.concatenate((iso20,dummy[None,...]),axis=0)
    if i % 50 == 0: print('time step: ' + str(i))

# remove first empty array
iso20 = iso20[1:,::]

# create xarray
thermocline_isas15 = xr.DataArray(iso20,coords={"time": argo.time.values,
                           "lon": (('lon'),argo.longitude),
                           "lat": (('lat'),argo.latitude)},
             dims=["time", "lat","lon"])

thermocline_isas15.to_dataset(name='therm20').to_netcdf('/vortexfs1/share/clidex/data/obs/ARGO/ISAS15/isas15_therm20_int1m_30E_160E_40S_35N.nc')

In [ ]:
#############################################
# mean and std of thermocline IO
base=['2002','2012']

# time-mean map
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline_isas15.lon,thermocline_isas15.lat,
                   thermocline_isas15['therm20'].sel(time=slice(*base)).mean('time',skipna=True),vmin=0,vmax=250,
                   cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='depth [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

plt.savefig(plotsave + 'isas15_'+base[0]+'_'+base[1]+'_isotherm20_time_mean.png',dpi=300,bbox_inches='tight')


# standard deviation
fig,ax = plt.subplots(figsize=(10,4),subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
cc = ax.pcolormesh(thermocline_isas15.lon,thermocline_isas15.lat,
                   thermocline_isas15['therm20'].sel(time=slice(*base)).std('time',skipna=True),vmin=0,vmax=45,
                   cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
ax.coastlines(resolution='50m')
ax.add_feature(cartopy.feature.LAND, color='lightgray')
ax.set_extent([25,180,-45,30],crs=ccrs.PlateCarree())
plt.colorbar(cc,shrink=0.8,label='standard deviation [m]')
gl.ylabels_right = False
gl.yformatter = LATITUDE_FORMATTER
gl.xformatter = LONGITUDE_FORMATTER

# plt.savefig(plotsave + 'isas15_'+base[0]+'_'+base[1]+'_isotherm20_std.png',dpi=300,bbox_inches='tight')

# Plot Figure S4

Subplot for mean (left) and std (right) for Ishii, ISAS , hindcast (2002-2012) and hindcast (full time period)

In [ ]:
# load data
thermocline_ishii = xr.open_dataset('/vortexfs1/share/clidex/data/obs/Ishii/ishii_therm20_int1m_20E_180E_45S_35N.nc')
thermocline_isas15 = xr.open_dataset('/vortexfs1/share/clidex/data/obs/ARGO/ISAS15/isas15_therm20_int1m_30E_160E_40S_35N.nc')
thermocline = xr.open_dataset('/vortexfs1/share/clidex/data/ORCA/ORCA025.L46.LIM2vp.CFCSF6.JRA.XIOS2-K003.hindcast/'\
                                  'derived_fields/k003_therm20_int1m_180E_180W_50S_45N.nc')

# base for short period
base = ['2002','2012']

# figure settings
plt.rcParams.update({'font.size': 12})
fig,ax=plt.subplots(nrows=4,ncols=2,figsize=(7,9),
                  subplot_kw = dict(projection=ccrs.PlateCarree(central_longitude=120)))
plt.subplots_adjust(bottom=0.1, top=0.9, left=0.03, right=0.97, wspace=0.04, hspace=0.19)

# loop through datasets for plotting
for ds,i in zip([thermocline_ishii,thermocline_isas15,thermocline],range(3)): 
    
    if i<2:
        cc = ax[i,0].pcolormesh(ds.lon,ds.lat,ds['therm20'].sel(time=slice(*base)).mean('time',skipna=True),
                           vmin=0,vmax=250,cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
        cc1 = ax[i,1].pcolormesh(ds.lon,ds.lat,ds['therm20'].sel(time=slice(*base)).std('time',skipna=True),
                           vmin=0,vmax=45,cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
    elif i==2:
        # short baseline
        # mask areas in Maritime Continent where isotherm not always present
        dummy = thermocline['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=False).values
        mask = np.zeros(thermocline['nav_lon'].shape)
        wo = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-22) & (thermocline['nav_lon'].values<150) & 
                       (thermocline['nav_lat'].values<25))
        wo2 = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-24) & (thermocline['nav_lon'].values>=140) & 
                       (thermocline['nav_lon'].values<=155) & (thermocline['nav_lat'].values<25))
        mask[wo] = 1000
        mask[wo2] = 1000
        cc = ax[i,0].pcolormesh(ds.nav_lon,ds.nav_lat,ds['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=True).where(mask<1000),
                           vmin=0,vmax=250,cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
        cc1 = ax[i,1].pcolormesh(ds.nav_lon,ds.nav_lat,ds['therm20'].sel(time_counter=slice(*base)).std('time_counter',skipna=True).where(mask<1000),
                           vmin=0,vmax=45,cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
        
        # full time period
        base = ['1960',None]
        # mask areas in Maritime Continent where isotherm not always present
        dummy = thermocline['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=False).values
        mask = np.zeros(thermocline['nav_lon'].shape)
        wo = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-22) & (thermocline['nav_lon'].values<150) & 
                       (thermocline['nav_lat'].values<25))
        wo2 = np.where((np.isnan(dummy)) & (thermocline['nav_lat'].values>-24) & (thermocline['nav_lon'].values>=140) & 
                       (thermocline['nav_lon'].values<=155) & (thermocline['nav_lat'].values<25))
        mask[wo] = 1000
        mask[wo2] = 1000
        cc = ax[i+1,0].pcolormesh(ds.nav_lon,ds.nav_lat,ds['therm20'].sel(time_counter=slice(*base)).mean('time_counter',skipna=True).where(mask<1000),
                           vmin=0,vmax=250,cmap=plt.get_cmap('Spectral',20),transform=ccrs.PlateCarree())
        cc1 = ax[i+1,1].pcolormesh(ds.nav_lon,ds.nav_lat,ds['therm20'].sel(time_counter=slice(*base)).std('time_counter',skipna=True).where(mask<1000),
                           vmin=0,vmax=45,cmap=plt.get_cmap('RdYlBu_r',20),transform=ccrs.PlateCarree())
        
# row labels
ax[0,0].text(1, 1.07, 'Ishii (2002-2012)', transform=ax[0,0].transAxes, 
                size=11, weight='bold',horizontalalignment='center',bbox=dict(facecolor='w',edgecolor='k'))
ax[1,0].text(1, 1.07, 'ISAS 15 (2002-2012)', transform=ax[1,0].transAxes, 
                size=11, weight='bold',horizontalalignment='center',bbox=dict(facecolor='w',edgecolor='k'))
ax[2,0].text(1, 1.07, 'hindcast (2002-2012)', transform=ax[2,0].transAxes, 
                size=11, weight='bold',horizontalalignment='center',bbox=dict(facecolor='w',edgecolor='k'))
ax[3,0].text(1, 1.07, 'hindcast (1960-2016)', transform=ax[3,0].transAxes, 
                size=11, weight='bold',horizontalalignment='center',bbox=dict(facecolor='w',edgecolor='k'))

# grid lines etc
for ax,i in zip(ax.flat,range(8)):  
    gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,
                           xlocs=[40,80,120,160,200,-160],ylocs=range(-60,60,30))
    ax.coastlines(resolution='50m')
    ax.add_feature(cartopy.feature.LAND, color='lightgray')
    ax.set_extent([25,180,-45,31],crs=ccrs.PlateCarree())
#     plt.colorbar(cc,shrink=0.8,label='standard deviation [m]')
    if i in [0,1,2,3,4,5]: gl.xlabels_bottom = False
    gl.xlabels_top = False #if i in [2,3,4,5,6,7]: 
    if i in [0,2,4,6]: gl.ylabels_right = False
    if i in [1,3,5,7]: gl.ylabels_left = False
#         if i in [2,4,5,6]: gl.xlabels_top = False
    gl.yformatter = LATITUDE_FORMATTER
    gl.xformatter = LONGITUDE_FORMATTER
    t = ax.text(0.02, 0.9, string.ascii_lowercase[i]+')', transform=ax.transAxes, 
                size=11, weight='bold')
    t.set_bbox(dict(facecolor='w',boxstyle='square,pad=0.2'))
    
# add colorbar
cbaxes = fig.add_axes([0.1, 0.03, 0.35, 0.02]) 
plt.colorbar(cc, cax = cbaxes,extend='max',label='20\N{DEGREE SIGN} isotherm depth [m]',orientation='horizontal') 
cbaxes = fig.add_axes([0.55, 0.03, 0.35, 0.02]) 
plt.colorbar(cc1, cax = cbaxes,extend='max',label='standard deviation [m]',orientation='horizontal')

# save figure
# finished_plot(fig,plotsave + 'SI_isotherm20_map_meandepth_std.png')